In [6]:
import datetime
import os.path
import requests
import googlemaps
import json

In [2]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [4]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

TOKEN_FILE = "token.json"
CREDENTIAL_FILE = "credential.json"

def main():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIAL_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        print('Getting the upcoming 10 events')
        events_result = service.events().list(calendarId='primary', timeMin=now,
                                              maxResults=10, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

        # Prints the start and name of the next 10 events
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])

    except HttpError as error:
        print('An error occurred: %s' % error)


if __name__ == '__main__':
    main()

Getting the upcoming 10 events
2022-11-03 Stay at Hampton Inn & Suites Seattle-Downtown


In [8]:
GOOGLE_MAP_KEY = "ENTER_KEY"

url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={GOOGLE_MAP_KEY}'
data = {
    'considerIp': True,
}

result = requests.post(url, data)

print(result.text)
result2 = json.loads(result.text)

lat = result2["location"]["lat"]
lng = result2["location"]["lng"]

gmaps = googlemaps.Client(GOOGLE_MAP_KEY)
reverse_geocode_result = gmaps.reverse_geocode((lat, lng),language='ko')

print(reverse_geocode_result)

{
  "location": {
    "lat": 36.0152785,
    "lng": 129.3309904
  },
  "accuracy": 1486.4628454273964
}

[{'address_components': [{'long_name': '７８０−１６１', 'short_name': '７８０−１６１', 'types': ['premise']}, {'long_name': '대이동', 'short_name': '대이동', 'types': ['political', 'sublocality', 'sublocality_level_2']}, {'long_name': '남구', 'short_name': '남구', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': '포항시', 'short_name': '포항시', 'types': ['locality', 'political']}, {'long_name': '경상북도', 'short_name': '경상북도', 'types': ['administrative_area_level_1', 'political']}, {'long_name': '대한민국', 'short_name': 'KR', 'types': ['country', 'political']}, {'long_name': '790-310', 'short_name': '790-310', 'types': ['postal_code']}], 'formatted_address': '대한민국 경상북도 포항시 남구 대이동 780-161', 'geometry': {'location': {'lat': 36.0150377, 'lng': 129.3308548}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 36.0163866802915, 'lng': 129.3322037802915}, 'southwest': {'lat': 36.0136